In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

# Carrega o dataset
try:
    df = pd.read_csv('housing_sp_city.csv', encoding='latin-1')
except FileNotFoundError:
    print("Erro: Arquivo 'housing_sp_city.csv' não encontrado.")
    exit()

df['bairro'] = df.bairro.astype('category')
df['tipo_imovel'] = df.tipo_imovel.astype('category')

# Filtra o DataFrame
df_filtered = df[df['tipo_anuncio'] == 'Venda']

# Seleciona as features e a variável alvo
try:
    X = df_filtered[['bairro', 'tipo_imovel', 'quartos', 'banheiros', 'vagas_garagem', 'area_util']]
    y = df_filtered['preco_venda']
except KeyError as e:
    print(f"Erro: Coluna '{e}' não encontrada no dataset. Verifique o nome das colunas.")
    exit()


# Divide os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializa e treina o modelo XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', enable_categorical=True, random_state=42) # Define o objetivo como regressão
model.fit(X_train, y_train)

# Faz previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avalia o modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Save the trained model, encoder, and imputer to pickle files
filename_model = 'housing_price_model_xgb.pkl'
pickle.dump(model, open(filename_model, 'wb'))